In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('../data/full_dataset.csv')

In [4]:
df

,Unnamed: 0,category,resume,job_title
0,0,data science,Skills * Programming Languages: Python (pandas...,NaN
1,1,data science,Education Details \r\nMay 2013 to May 2017 B.E...,NaN
2,2,data science,"Areas of Interest Deep Learning, Control Syste...",NaN
3,3,data science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,NaN
4,4,data science,"Education Details \r\n MCA YMCAUST, Faridab...",NaN
...,...,...,...,...
15352,2491,automobile,GENERAL LIABILITY CLAIM REPRESENTATIVE\r\nSumm...,NaN
15353,2492,automobile,AUTOMOBILE TRANSPORTER\r\nProfessional Summary...,NaN
15354,2493,automobile,"Highlights\r\n\r\nProg. Languages: C (5+ yrs),...",NaN
15355,2494,automobile,CUSTOMER RELATIONS SPECIALIST\r\nSummary\r\n\r...,NaN
